In [ ]:
#pip install --upgrade --user ortools

In [ ]:
from ortools.sat.python import cp_model

In [ ]:
# Variables

staff_name = ['A_SAC', 'B_SAC', 'C_SAC', 'D_SAC', 'A_AC1', 'A_AC2', 'A_AC3', 'B_AC1', 'B_AC2', 'B_AC3'] # 10 staff

shifts_name = ['Morning', 'Afternoon', 'Night']  #0: Morning, 1: Afternoon, 2: Night

days = ['Mon', 'Tue', 'Wed', 'Thur', 'Fri', 'Sat', 'Sun'] # 7 days in a week

# Change to number for loop

all_staff = range(len(staff_name))
all_shifts = range(len(shifts_name))
all_days = range(len(days))

In [ ]:
all_staff

range(0, 10)

In [ ]:
all_shifts

range(0, 3)

In [ ]:
all_days

range(0, 7)

In [ ]:
# Model = Constraint Programming

model = cp_model.CpModel()

In [ ]:
# Create Outcome Variables 
# The array defines assignments for shifts to nurses as follows:
# shifts[(n, d, s)] equals 1 if shift s is assigned to staff n on day d, and 0 otherwise.

shifts = {}
for n in all_staff:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

Constrainst Definition

2 types of shifts:

1.   No-Working Shift, O
2.   Working Shift:


> ● Morning shifts: 1M, 2M

> ● Afternoon shifts: 1A, 2A

> ● Night shifts: 1N, 2N



In [ ]:
# Constraint
# Each shift is assgined to >=1 staff per day

for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts[(n, d, s)] for n in all_staff) >= 1)

In [ ]:
# Constraint
# Each staff only works <= 1 shift per day

for n in all_staff:
    for d in all_days:
        model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)

In [ ]:
# Constraint:
# Each Staff only <= 2 Consecutive Same Shifts

for n in all_staff:
    for d in range(len(days) - 1):
        model.Add(sum(shifts[(n, d, s)] + shifts[(n, d + 1, s)] for s in all_shifts) <= 2)



Staffs can only receive several Shifts due to their skillsets, 
i.e, Staff C_SAC can only receive shifts 1A, 2A, and 1M, 2M

In [ ]:
# Constraint:
# Staff C_SAC cannot receive Night Shift
# Staff C_SAC: n = 2, Night Shift s != 2 for all day

#for s in all_shifts:
for d in range(len(days)):
  model.Add(sum(shifts[(2, d, s)] * s for s in all_shifts ) != 2)



Staff C_SAC must take one shift type (Afternoon or  Morning) in 2 consecutive days, meaning that for a window of 3 days, he cannot take off day on 2nd day (1) and  he has to work either d = 1 or d = 3 (2) and same shift type on 2nd day (3)

1.   shifts (n = 2, d = 1, s) = 1
2.   shifts (n = 2, d = 0 , s ) + shifts (n = 2, d = 2, s) = 1
3. shifts (n = 2, d = 1, s) * s = [shifts (n = 2, d = 0 , s ) + shifts (n = 2, d = 2, s)] * s







In [ ]:
# Constraints:

# Window 1: [day = 0 to 2]
model.Add(sum(shifts[(2, 1, s)]  for s in all_shifts) == 1)
model.Add(sum(shifts[(2, 0, s)] + shifts[(2, 2, s)] for s in all_shifts) == 1)
model.Add(sum(shifts[(2, 1, s)] * s for s in all_shifts) == sum(shifts[(2, 0, s)] + shifts[(2, 2, s)] * s for s in all_shifts) )


# Window 2: [day = 3 to 5]
#model.Add(sum(shifts[(2, 4, s)]  for s in all_shifts) == 1)
#model.Add(sum(shifts[(2, 3, s)] + shifts[(2, 5, s)] for s in all_shifts) == 1)
#model.Add(sum(shifts[(2, 4, s)] * s for s in all_shifts) == sum(shifts[(2, 3, s)] + shifts[(2, 5, s)] * s for s in all_shifts) )


# For window 2:If I set up this contraints I will violate distribute evenly thus there no solution for it.
# We have 7 * 3 = 21 shifts * 2 (max 2 staff per shift) = 42 shifts
# 10 staff = [21 - 42] shifts
# 1 staff = [2 - 4] shifts for each staff. If staff C_SAT take 2 days in the window 2 ---> not enough shifts to distribute shift evenly among 10 staff over 7 days

Maximum Total Working Hours that a staff can work. You can see the Actual Total working Hours recorded in Staff Statistics on the right panel, i.e Staff A_SAC totally spends 177 Working Hours

In [ ]:
# Constraint:
# Because we take the days = 7 and take assumption of maximum working hours for each staff is 5 * 8 = 40 hours and each shift = 8 hours
# Each staff only takes 40 hours per 7 days (every week) or 5 shifts per week

for n in all_staff:
    num_shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked.append(shifts[(n, d, s)])
    model.Add(sum(num_shifts_worked) <= 5)


Requirement on total number of staffs need to work Per Day, showing in Coverage Statistics at the bottom panel, i.e., we need At Least 1 staff for Afternoon shift (either 1A or 2A)

In [ ]:
# Constraint:
# Each day for Each shift, the number of staff assigned is <= 2 

for d in all_days:
    for s in all_shifts:
        model.Add(sum(shifts[(n, d, s)] for n in all_staff) <=2)

In [ ]:
# Constraint
# This constraint is met when 
# Each shift is assgined to >=1 staff per day
# and 
# Each Staff only <= 2 Consecutive Same Shifts
# No config needed for this constraint
 

Shift Patterns: there are some repeated patterns, i.e, 1M→2M→ 1N→ 2N, 1N→ 2N→ O

In [ ]:
# Constraint
# This is tricky. I will take the second constraint first
# Each staff must take one day off after 2 Night shift
# Night shift takes value of 2 in our confit in shifts --> we can take 3 consecutive days the sum of shift must less than 4
# It can interpreted as following:

# Because s can take value 0 and shifts can take value of 0 so if shift [0,1] * s[0,1,2] = 0. I cannot identify whether shift = 0 or s = 0 (Morning).
# Therefore, I need to multiply by (s + 1). Shift [0,1] * (s + 1) [1,2,3]
# For each staff, three consecutive day, the shifts in number must <= 6

for n in all_staff:
    for d in range(len(days) - 2):
        model.Add(sum(shifts[(n, d, s)] + shifts[(n, d + 1, s)] + shifts[(n, d + 2, s)] * (s + 1) for s in all_shifts) <= 6)


In [ ]:
# Constraint
# For the pattern 2 Morning Shift then 2 Night Shift
# For 5 consecutive days, if a staff take 2 Morning Shift in consecutive then this staff should take 2 night shift for next 2 consecutive days and then one day Off
# Because s can take value 0 and shifts can take value of 0
# Therefore, I need to multiply by (s + 1). Shift [0,1] * (s + 1) [1,2,3]
# For each staff, five consecutive day, the shifts in number must <= 8
# What if, Shift [0,1] * (s + 1) [1,2,3] = 2 for 4 consecutive days then the sum  will be = 8. 
# It means a staff can take 4 Afternoon Shift in 4 consecutive days. However we already defined constraints 
# Each Staff only <= 2 Consecutive Same Shifts so we can eliminate the 4 Afternoon Shift Each staff

for n in all_staff:
    for d in range(len(days) - 4):
        model.Add(sum(shifts[(n, d, s)] + shifts[(n, d + 1, s)] + shifts[(n, d + 2, s)] + shifts[(n, d + 3, s)] + shifts[(n, d + 4, s)]  * (s+1) for s in all_shifts) <= 8)

In [ ]:
# Constraint (this one I took from google sample )
# The last constrait is not stated in the assignment, however it is nice to have
# Try to distribute the shifts evenly, so that each nurse works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses, some nurses will
# be assigned one more shift.

num_shifts = 3
num_days = 7
num_staff = 10

min_shifts_per_staff = (num_shifts * num_days) // num_staff
if num_shifts * num_days % num_staff == 0:
    max_shifts_per_staff = min_shifts_per_staff
else:
    max_shifts_per_staff = min_shifts_per_staff + 1
for n in all_staff:
    num_shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_shifts_per_staff <= sum(num_shifts_worked))
    model.Add(sum(num_shifts_worked) <= max_shifts_per_staff)

In [ ]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

In [ ]:
# This is Google API, I changed a litle bit

class NursesPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_nurses, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_nurses = num_nurses
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print('Solution %i' % self._solution_count)
        for d in range(self._num_days):
            print('Day %i' % d)
            for n in range(self._num_nurses):
                is_working = False
                for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        print('  Staff %s works shift %s' % (staff_name[n], shifts_name[s]))
                if not is_working:
                    print('  Staff ' + staff_name[n] + ' does not work')
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

# Display the first five solutions.
solution_limit = 5
solution_printer = NursesPartialSolutionPrinter(shifts, num_staff, num_days, num_shifts, solution_limit)

In [ ]:
solver.Solve(model, solution_printer)

Solution 1
Day 0
  Staff A_SAC does not work
  Staff B_SAC does not work
  Staff C_SAC does not work
  Staff D_SAC does not work
  Staff A_AC1 works shift Morning
  Staff A_AC2 does not work
  Staff A_AC3 works shift Afternoon
  Staff B_AC1 works shift Afternoon
  Staff B_AC2 works shift Night
  Staff B_AC3 works shift Night
Day 1
  Staff A_SAC does not work
  Staff B_SAC does not work
  Staff C_SAC works shift Afternoon
  Staff D_SAC does not work
  Staff A_AC1 does not work
  Staff A_AC2 works shift Night
  Staff A_AC3 works shift Night
  Staff B_AC1 works shift Afternoon
  Staff B_AC2 works shift Morning
  Staff B_AC3 works shift Morning
Day 2
  Staff A_SAC does not work
  Staff B_SAC does not work
  Staff C_SAC works shift Afternoon
  Staff D_SAC works shift Night
  Staff A_AC1 does not work
  Staff A_AC2 works shift Morning
  Staff A_AC3 does not work
  Staff B_AC1 does not work
  Staff B_AC2 does not work
  Staff B_AC3 does not work
Day 3
  Staff A_SAC works shift Night
  Staff B

2